# Char RNN

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

## 1. Preprocessing Train data

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
print(char_vocab)

['!', 'a', 'e', 'l', 'p']


In [3]:
vocab_size = len(char_vocab)

We are going to use a `one hot vector` for every character. This means every character is going to have a `vocab_size` size of vector.

In [4]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [5]:
char_to_index = dict((c,i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [21]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [22]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [23]:
x_one_hot = [np.eye(vocab_size)[x_data]]
# y_data = [np.eye(vocab_size)[y_data]]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/var/folders/0r/6ncbx7693d96pr3xr0xpgt8w0000gn/T/ipykernel_17587/641410555.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_62dm4livko/croot/pytorch_1675190252673/work/torch/csrc/utils/tensor_new.cpp:204.)
  X = torch.FloatTensor(x_one_hot)


In [24]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [25]:
net = Net(input_size, hidden_size, output_size)

In [26]:
outputs = net(X)

In [27]:
print(outputs.shape)

torch.Size([1, 5, 5])


In [28]:
print(outputs.view(-1, input_size).shape)

torch.Size([5, 5])


In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs= net(X)
    loss = criterion(outputs.view(-1, 5), Y.view(-1))
    loss.backward()
    optimizer.step()

    result